In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pprint
import numpy as np
import pandas as pd
from pathlib import Path

from matplotlib import pyplot as plt
import seaborn as sns
import scanpy as sc

from sklearn.metrics import adjusted_rand_score
from IPython.display import display, HTML

import warnings
from numba.core.errors import NumbaDeprecationWarning

from calicost import arg_parse

In [ ]:
from sim_analysis import (
    get_config,
    get_sampleid,
    get_best_r_hmrf,
    get_rdrbaf,
    get_true_clones_path,
    get_true_clones,
    get_sim_runs,
    get_numbat_path,
    get_numbat_clones,
    get_starch_clones,
    plot_true_clones,
    get_calico_clones,
    read_gene_loc,
    read_true_gene_cna,
    read_calico_gene_cna,
    read_numbat_gene_cna,
    read_starch_gene_cna,
    compute_gene_F1,
    get_aris, 
    plot_aris,
    get_f1s,
    plot_f1s,
    plot_calico_clones
)

## Configuration

In [ ]:
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [ ]:
pd.set_option('display.max_rows', 10)
plt.rcParams.update({'font.size': 14})
sc.set_figure_params(dpi=120)

In [ ]:
! pwd

In [ ]:
true_dir = "../simulated_data_related"
calico_pure_dir = "../nomixing_calicost_related"
numbat_dir = "../numbat_related"
starch_dir = "../starch_related"

# hg_table_file = "/nfs/turbo/umms-congma1/projects/CalicoST/GRCh38_resources/hgTables_hg38_gencode.txt"
hg_table_file = "/Users/mw9568/Work/ragr/CalicoST/GRCh38_resources/hgTables_hg38_gencode.txt"

## Preface

Note:
 - min. 100 spots per clone
 - assumes (max.) of 3 clones

### Available Runs

In [ ]:
sim_runs = get_sim_runs()
sim_runs

### Config for a given run

In [ ]:
config_path = "../nomixing_calicost_related//numcnas1.2_cnasize1e7_ploidy2_random0/configfile0"

config = get_config(config_path)
pprint.pprint(config)

In [ ]:
rdrbaf = get_rdrbaf(config_path, 3, relative_path="../nomixing_calicost_related/", verbose=False)

# NB (# states x # clones TBC).
rdrbaf["new_log_mu"].shape, rdrbaf["new_alphas"].shape, rdrbaf["new_p_binom"].shape, rdrbaf['log_gamma'].shape, rdrbaf['total_llf']

In [ ]:
random_state = 9

# state = [(3, 3), "3e7", 2, random_state]
state = [(1, 2), "1e7", 2, random_state]

true = plot_true_clones(true_dir, *state)
est = plot_calico_clones(calico_pure_dir, *state, true_dir)

# Clone identification accuracy

In [ ]:
calico_clones = get_calico_clones(calico_pure_dir, *state, true_dir=true_dir)
calico_clones

In [ ]:
starch_clones = get_starch_clones(starch_dir, get_sampleid((3, 3), "3e7", 2, 0), true_clones=true_clones)
starch_clones

In [ ]:
clone_aris = get_aris(true_dir, calico_pure_dir, numbat_dir, starch_dir)
clone_aris

In [ ]:
plot_aris(clone_aris)

# Event detection accuracy

The detection accuracy is evalated on a per-gene level:
the precision & sensitivity of genes involved in each category of event (Deletion, Amplication, LOH) for all events.

In [ ]:
# NB (chr, start, end) for a given gene list.
df_hgtable = read_gene_loc(hg_table_file)
df_hgtable

In [ ]:
df_event_f1 = get_f1s(true_dir, df_hgtable, calico_pure_dir, numbat_dir, starch_dir)
df_event_f1

In [ ]:
plot_f1s(df_event_f1)

# Done.